In [1]:
"""
Prerequisites 

Deep-translator 1.8.3 
https://pypi.org/project/deep-translator/

pip install -U deep-translator

"""

import os
import pandas as pd
import numpy as np
from deep_translator import GoogleTranslator

pd.set_option('display.max_rows', None)

#Print data frame rows, default 5
def print_data(data,rows=5):
    display(data.head(rows)) 
    
#Load the json file    
def load_json_data():
    df = pd.read_json('supplier_car.json') 
    return df  

# Clean (drop 'entity_id') and sort data by 'ID','Attribute Names'
def clean_sort_data(data):
    df = data.drop(['entity_id'], axis=1).sort_values(['ID','Attribute Names']).reset_index()
    return df

#Explore data. Check for unique values
def explore_data(data):
    print_data(data)
    display(data['ID'].unique().size)
    display(data['Attribute Names'].unique().size)
    display(data.shape)
    
# Split data into two dataframes. Use pivot to reshape data    
def split_pivot_data(data):  
    df1 = data[['ID','MakeText','TypeName','TypeNameFull','ModelText','ModelTypeText']].drop_duplicates().set_index('ID')
        
    df2 = data.pivot(index='ID', 
                     columns='Attribute Names', 
                     values='Attribute Values').reset_index().rename_axis(None, axis=1).set_index('ID')
    
    return df1, df2   

#Merge two data sets on 'ID'
def merge_data(dx,dy): 
    data = dx.merge(dy, on='ID')
    return data

#Map supplier data to target data
def map_to_target_data(data):
    return data[
    ['BodyTypeText',
     'BodyColorText',
     'ConditionTypeText',
     'City',
     'MakeText',
     'FirstRegYear',
     'Km',
     'ModelText',
     'ModelTypeText',
     'FirstRegMonth',
    'ConsumptionTotalText']].rename(columns={
                      'BodyTypeText' :'carType',
                      'ConditionTypeText':'condition',
                      'BodyColorText':'color',
                      'City':'city',
                      'MakeText':'make',
                      'FirstRegYear':'manufacture_year',
                      'Km':'mileage',
                      'ModelText':'model',
                      'ModelTypeText':'model_variant',
                      'FirstRegMonth':'manufacture_month',
                      'ConsumptionTotalText':'fuel_consumption_unit'})  

#Translate value using GoogleTranslator. Return dictonary as de=>en
def translate(data, column):
    items_de = data[~data[column].isna()][column].unique()
    items = {}
    for x in items_de:
        items[x] = GoogleTranslator(source='de', target='en').translate(x).capitalize()
    return items

# Make the normalization steps. Load translations into dictonary as de=>en. Use lamda function for bulk inline transformation
def normalize_data(data):
    
    colors = translate(data,'color')
    carTypes = translate(data,'carType')
    conditions = translate(data,'condition')
    
    data['color']=data['color'].apply(lambda x: colors[x] if colors.get(x) != None else x)
    data['carType']=data['carType'].apply(lambda x: carTypes[x] if carTypes.get(x) != None else x)
    data['condition']=data['condition'].apply(lambda x: conditions[x] if conditions.get(x) != None else x)
    data['fuel_consumption_unit']=data['fuel_consumption_unit'].apply(lambda x: 'l_km_consumption' if 'l/100km' in x else x)

    return data  

#Fill the missing columns to achive the same schema as target
def integrate_data(data):
    
    data.insert(3, 'currency', 'null')
    data.insert(4, 'drive', 'null')
    data.insert(6, 'country', 'null')
    data.insert(10, 'mileage_unit', 'null')
    data.insert(13, 'price_on_request', 'null')
    data.insert(14, 'type', 'null')
    data.insert(15, 'zip', 'null')
    
    return data

# Output data frame to excel file
def output_to_excel(file, data_x, sheet):
    mode='w'
    if os.path.exists(file):
         mode='a'
        
    with pd.ExcelWriter(file, engine='openpyxl', mode=mode) as writer:  
        data_x.to_excel(writer, sheet_name=sheet, index=False)

<h1>Introduction</h1>

An e-commerce shop would like to onboard new suppliers efficiently. To enable the onboarding process, the customer needs us to integrate product data from suppliers in various formats and
styles into the pre-defined data structure of their e-commerce shop application.

The goal is to transform the supplier data so that it could be directly loaded into the target dataset without any other changes.

Json file is loaded into data frame. An empty excel file is created for saving data for each step.

In [2]:
if __name__ == '__main__':
    
    data = load_json_data()
    print_data(data)
    display(data.shape)
    
    file = 'Integrated_supplier_data.xlsx'
    if os.path.exists(file):
         os.remove(file)   

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
0,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed
1,1059,MERCEDES-BENZ,ML 350 Inspiration,MERCEDES-BENZ ML 350 Inspiration,ML 350,ML 350 Inspiration,Hp,235,00107c2d-0071-4475-88f0-810133638b7e
2,524,AUDI,S6 Avant quattro 4.2,AUDI S6 Avant quattro 4.2,S6,S6 Avant quattro 4.2,FuelTypeText,Benzin,00126794-a8ef-48fe-93d6-43cfc69fbfb6
3,608,SAAB,9-3 2.0i-16 TS Aero,SAAB 9-3 2.0i-16 TS Aero,9-3,9-3 2.0i-16 TS Aero,Ccm,1985,00182529-1bf7-4f93-89fa-2e8e634b2c9d
4,726,PORSCHE,911 Turbo Cabrio,PORSCHE 911 Turbo Cabrio,911,911 Turbo Cabrio,BodyColorText,schwarz mét.,002d30c2-43f6-4905-868f-160dbc445c56


(21906, 9)

<h1>1. Pre-processing</h1>

The provided supplier data have 21906 rows and 9 columns (21906 x 9). Atributes are listed as key=>values pairs in two columns. 

In [3]:
    #Clean and sort the data
    data_cleaned = clean_sort_data(data)
    explore_data(data_cleaned)

,index,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values
0,10902,1,MERCEDES-BENZ,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,E 320,E 320 Elégance 4-Matic,BodyColorText,anthrazit
1,7573,1,MERCEDES-BENZ,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,E 320,E 320 Elégance 4-Matic,BodyTypeText,Limousine
2,19538,1,MERCEDES-BENZ,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,E 320,E 320 Elégance 4-Matic,Ccm,3199
3,5931,1,MERCEDES-BENZ,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,E 320,E 320 Elégance 4-Matic,City,Zuzwil
4,21164,1,MERCEDES-BENZ,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,E 320,E 320 Elégance 4-Matic,Co2EmissionText,275 g/km


1153

19

(21906, 9)

In [4]:
    #Find the Model Id that have 18 attributes
    atributes_count = data.groupby(['ID'])['ID'].count().sort_values()
    print_data(atributes_count)

ID
824    18
1      19
771    19
770    19
769    19
Name: ID, dtype: int64

The data set have 1153 uniques ID's and 19 unique Atributes. If we multiply 1153 with 19 we get 21907. This means that 1152 model ID's have 19 atributes and one model with ID (824) have 18 atributes. Having this on mind we can conclude that the data set is almost a perfect matrix (1153 x 19).

In the Target sheet each model atribute represet a unique column, while in Supplier the atributes are listed as key=>values pairs in two columns: Attribute Names and Attribute Values.

In order to achieve the same granularity as the target data the supplier data needs to be splited into two data sets:

1. One with colums: 'ID','MakeText','TypeName','TypeNameFull','ModelText','ModelTypeText'

2. Second using pivot to reshape the given data frame based on Atributes Values and Names.

In [5]:
    #Split original data set in two data set 
    dx,dy = split_pivot_data(data_cleaned)
    print_data(dx)
    display(dx.shape)
    print_data(dy)
    display(dy.shape)

,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText
ID,,,,,
1,MERCEDES-BENZ,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,E 320,E 320 Elégance 4-Matic
2,AUDI,RS6 Avant 5.0 V10 quattro,AUDI RS6 Avant 5.0 V10 quattro,RS6,RS6 Avant 5.0 V10 quattro
3,AUDI,RS6 Avant quattro,AUDI RS6 Avant quattro,RS6,RS6 Avant quattro
4,CHEVROLET,Corvette Z06,CHEVROLET Corvette Z06,CORVETTE,Corvette Z06
5,PORSCHE,Cayenne Turbo Techart Magnum Kit,PORSCHE Cayenne Turbo Techart Magnum Kit,CAYENNE,Cayenne Turbo Techart Magnum Kit


(1153, 5)

,BodyColorText,BodyTypeText,Ccm,City,Co2EmissionText,ConditionTypeText,ConsumptionRatingText,ConsumptionTotalText,Doors,DriveTypeText,FirstRegMonth,FirstRegYear,FuelTypeText,Hp,InteriorColorText,Km,Properties,Seats,TransmissionTypeText
ID,,,,,,,,,,,,,,,,,,,
1,anthrazit,Limousine,3199,Zuzwil,275 g/km,Occasion,null,11.5 l/100km,4,Allrad,1,1999,Benzin,224,grau,31900,"""Ab MFK""",5,Automat
2,anthrazit,Kombi,4991,Zuzwil,333 g/km,Occasion,G,14.0 l/100km,5,Allrad,7,2008,Benzin,580,null,25400,"""Ab MFK""",5,Automat sequentiell
3,anthrazit,Kombi,4172,Zuzwil,350 g/km,Occasion,G,14.6 l/100km,5,Allrad,10,2002,Benzin,450,beige,38500,"""Ab MFK""",5,Automat sequentiell
4,anthrazit,Coupé,6162,Zuzwil,291 g/km,Occasion,G,12.7 l/100km,2,Hinterradantrieb,6,2015,Benzin,660,schwarz,200,"""Ab MFK"", ""Direkt-/Parallelimport""",2,Schaltgetriebe manuell
5,anthrazit,SUV / Geländewagen,4806,Zuzwil,270 g/km,Occasion,G,11.5 l/100km,5,Allrad,1,2010,Benzin,500,schwarz,2900,"""Ab MFK""",5,Automat sequentiell


(1153, 19)

After splitting and pivoting the supplier data we have now two data frames with shape (1153 x 5) and (1153 x 19). We can now merge them (by ID) into one dataframe.

In [6]:
    #Merge two data sets into one
    pre_procesed_data = merge_data(dx,dy)
    print_data(pre_procesed_data)
    display(pre_procesed_data.shape)
    output_to_excel(file, pre_procesed_data,'1. Pre-processing')

,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,BodyColorText,BodyTypeText,Ccm,City,Co2EmissionText,...,DriveTypeText,FirstRegMonth,FirstRegYear,FuelTypeText,Hp,InteriorColorText,Km,Properties,Seats,TransmissionTypeText
ID,,,,,,,,,,,,,,,,,,,,,
1,MERCEDES-BENZ,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,E 320,E 320 Elégance 4-Matic,anthrazit,Limousine,3199,Zuzwil,275 g/km,...,Allrad,1,1999,Benzin,224,grau,31900,"""Ab MFK""",5,Automat
2,AUDI,RS6 Avant 5.0 V10 quattro,AUDI RS6 Avant 5.0 V10 quattro,RS6,RS6 Avant 5.0 V10 quattro,anthrazit,Kombi,4991,Zuzwil,333 g/km,...,Allrad,7,2008,Benzin,580,null,25400,"""Ab MFK""",5,Automat sequentiell
3,AUDI,RS6 Avant quattro,AUDI RS6 Avant quattro,RS6,RS6 Avant quattro,anthrazit,Kombi,4172,Zuzwil,350 g/km,...,Allrad,10,2002,Benzin,450,beige,38500,"""Ab MFK""",5,Automat sequentiell
4,CHEVROLET,Corvette Z06,CHEVROLET Corvette Z06,CORVETTE,Corvette Z06,anthrazit,Coupé,6162,Zuzwil,291 g/km,...,Hinterradantrieb,6,2015,Benzin,660,schwarz,200,"""Ab MFK"", ""Direkt-/Parallelimport""",2,Schaltgetriebe manuell
5,PORSCHE,Cayenne Turbo Techart Magnum Kit,PORSCHE Cayenne Turbo Techart Magnum Kit,CAYENNE,Cayenne Turbo Techart Magnum Kit,anthrazit,SUV / Geländewagen,4806,Zuzwil,270 g/km,...,Allrad,1,2010,Benzin,500,schwarz,2900,"""Ab MFK""",5,Automat sequentiell


(1153, 24)

The shape of merged data is (1153 x 24). We save the Pre-processed data into excel file.

# 2. Normalization

First we map the pre-pocessed data with the target data. 

In [7]:
    #Map pre-processed data to the target data
    mapped_data = map_to_target_data(pre_procesed_data)
    print_data(mapped_data)
    display(mapped_data.shape)

,carType,color,condition,city,make,manufacture_year,mileage,model,model_variant,manufacture_month,fuel_consumption_unit
ID,,,,,,,,,,,
1,Limousine,anthrazit,Occasion,Zuzwil,MERCEDES-BENZ,1999,31900,E 320,E 320 Elégance 4-Matic,1,11.5 l/100km
2,Kombi,anthrazit,Occasion,Zuzwil,AUDI,2008,25400,RS6,RS6 Avant 5.0 V10 quattro,7,14.0 l/100km
3,Kombi,anthrazit,Occasion,Zuzwil,AUDI,2002,38500,RS6,RS6 Avant quattro,10,14.6 l/100km
4,Coupé,anthrazit,Occasion,Zuzwil,CHEVROLET,2015,200,CORVETTE,Corvette Z06,6,12.7 l/100km
5,SUV / Geländewagen,anthrazit,Occasion,Zuzwil,PORSCHE,2010,2900,CAYENNE,Cayenne Turbo Techart Magnum Kit,1,11.5 l/100km


(1153, 11)

We were able to map 11 out of 18 columns (attributes). 

<p>
<li>'BodyTypeText' =>'carType'</li>
<li>'ConditionTypeText'=>'condition'</li>
<li>'BodyColorText'=>'color'</li>
<li>'City'=>'city'</li>
<li>'MakeText'=>'make'</li>
<li>'FirstRegYear'=>'manufacture_year'</li>
<li>'Km'=>'mileage'</li>
<li>'ModelText'=>'model'</li>
<li>'ModelTypeText'=>'model_variant'</li>
<li>'FirstRegMonth'=>'manufacture_month'</li>
<li>'ConsumptionTotalText'=>'fuel_consumption_unit'</li>
</p>

In [8]:
    #Normalize known column values
    normalized_data = normalize_data(mapped_data)
    print_data(normalized_data)
    display(normalized_data.shape)

,carType,color,condition,city,make,manufacture_year,mileage,model,model_variant,manufacture_month,fuel_consumption_unit
ID,,,,,,,,,,,
1,Limousine,Anthracite,Occasion,Zuzwil,MERCEDES-BENZ,1999,31900,E 320,E 320 Elégance 4-Matic,1,l_km_consumption
2,Station wagon,Anthracite,Occasion,Zuzwil,AUDI,2008,25400,RS6,RS6 Avant 5.0 V10 quattro,7,l_km_consumption
3,Station wagon,Anthracite,Occasion,Zuzwil,AUDI,2002,38500,RS6,RS6 Avant quattro,10,l_km_consumption
4,Coupe,Anthracite,Occasion,Zuzwil,CHEVROLET,2015,200,CORVETTE,Corvette Z06,6,l_km_consumption
5,Suv / suv,Anthracite,Occasion,Zuzwil,PORSCHE,2010,2900,CAYENNE,Cayenne Turbo Techart Magnum Kit,1,l_km_consumption


(1153, 11)

We normalize the attributes values for color, car_type, condition and fuel_consumption_unit. First three needs to be translated from German to English. For last one we search for spesific key word: l/100km.

For translation we use Deep Translator tool https://pypi.org/project/deep-translator/. For beter performance we translate unique column values and keep the results in dictonaries.

Not normalized columns values are:
<ul>
<li>'currency'</li>
<li>'drive'</li>
<li>'country'</li>
<li>'mileage_unit'</li>
<li>'price_on_request'</li>
<li>'type'</li>
<li>'zip'</li>
</ul>    

These column values can be autopopulate in we use some API's where from provided City Name we can get information about it. 
For example: https://github.com/DenisCarriere/geocoder
<ul>
<li>'currency'</li>
<li>'country'</li>
<li>'zip'</li>
</ul>


For there column values we need exact values because we cannot auto identify them. For example a car in Switzerland can have Rigth hand drive and miles as mileage unit.

<ul>
<li>'drive'</li>
<li>'mileage_unit'</li>
<li>'price_on_request'</li>
<li>'type'</li>
</ul>


In [9]:
    output_to_excel(file, normalized_data,'2. Normalization')

The shape of merged data is (1153 x 11). We save the Normalized data into excel file (second sheet).

# 3. Integration  


In this step we transform the Supplier Data into a new dataset thah have the same data schema as Target Data.

Not normalized or mapped columns are insert into corect position with null values.
<ul>
<li>'currency'</li>
<li>'drive'</li>
<li>'country'</li>
<li>'mileage_unit'</li>
<li>'price_on_request'</li>
<li>'type'</li>
<li>'zip'</li>
</ul>    

In [10]:
    # Transform the supplier data into target data schema
    integrated_data = integrate_data(normalized_data)
    print_data(integrated_data)
    display(integrated_data.shape)

,carType,color,condition,currency,drive,city,country,make,manufacture_year,mileage,mileage_unit,model,model_variant,price_on_request,type,zip,manufacture_month,fuel_consumption_unit
ID,,,,,,,,,,,,,,,,,,
1,Limousine,Anthracite,Occasion,null,null,Zuzwil,null,MERCEDES-BENZ,1999,31900,null,E 320,E 320 Elégance 4-Matic,null,null,null,1,l_km_consumption
2,Station wagon,Anthracite,Occasion,null,null,Zuzwil,null,AUDI,2008,25400,null,RS6,RS6 Avant 5.0 V10 quattro,null,null,null,7,l_km_consumption
3,Station wagon,Anthracite,Occasion,null,null,Zuzwil,null,AUDI,2002,38500,null,RS6,RS6 Avant quattro,null,null,null,10,l_km_consumption
4,Coupe,Anthracite,Occasion,null,null,Zuzwil,null,CHEVROLET,2015,200,null,CORVETTE,Corvette Z06,null,null,null,6,l_km_consumption
5,Suv / suv,Anthracite,Occasion,null,null,Zuzwil,null,PORSCHE,2010,2900,null,CAYENNE,Cayenne Turbo Techart Magnum Kit,null,null,null,1,l_km_consumption


(1153, 18)

We have now a data frame that have the same schema as Target Data (18 columns).

In [11]:
output_to_excel(file, integrated_data,'3. Integration')

We save the Integrated data into excel file (third sheet).